In [ ]:
### I/O SimpleITK

# import SimpleITK as sitk
# avmlabel = sitk.ReadImage("Data\AVL-LR-Final-Case53\S-Final-1-label.nrrd")
# vnda = sitk.GetArrayFromImage(lungim)
# fvess = sitk.GetImageFromArray(vnds)

In [1]:
import numpy as np
import nrrd  # pip install pynrrd
from skimage.morphology import skeletonize_3d  # pip install scikit-image
import networkx as nx

In [2]:
# Load/create the 3D mask

avmlabel, metadata = nrrd.read("z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-47-0.5_QR40_RESEARCH_A_80KV_Qr40d_3_65177004/Markups/88-Nathan/S-Final-1-label.nrrd")

atree = np.zeros_like(avmlabel)
atree[avmlabel == 1] = 1

In [3]:
# Create and save the skeleton

skeleton = skeletonize_3d(atree.astype(np.uint8))
nrrd.write("z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-47-0.5_QR40_RESEARCH_A_80KV_Qr40d_3_65177004/Markups/88-Nathan/E-Skeleton-1.nrrd", skeleton)

# np.shape(np.nonzero(skeleton)) # (3, 9862)
# np.nonzero(skeleton)
# indices = np.transpose(np.nonzero(skeleton))
# np.transpose(np.nonzero(skeleton))

In [5]:
skeleton, _ = nrrd.read("z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-47-0.5_QR40_RESEARCH_A_80KV_Qr40d_3_65177004/Markups/88-Nathan/E-Skeleton-1.nrrd")

In [ ]:
### Old: Create Graph/Neighbours/...

# np.shape(avmlabel)
# skeleton[45:52,263:270,198:204]

import networkx as nx

# Convert skeleton to graph
graph = nx.Graph()
indices = np.transpose(np.nonzero(skeleton))
graph.add_nodes_from(map(tuple, indices[0:20]))

# # Add edges to the graph
# for index in indices:
#     neighbors = np.array(list(nx.generate_adjlist(graph, [tuple(index)])))[0].astype(int)
#     for neighbor in neighbors:
#         graph.add_edge(tuple(index), tuple(np.array(list(map(int, neighbor.split(','))))))

# # Simplify the graph (remove intermediate nodes and consolidate segments)
# simplified_graph = graph.copy()
# # ... You need to implement the simplification process based on your criteria.


In [5]:
# Function: Print Nodes/Edges/...

def print_stats(graph):
    num_nodes = nx.number_of_nodes(graph)
    num_edges = nx.number_of_edges(graph)
    all_nodes = list(graph.nodes)
    all_edges = list(graph.edges)

    print("Number of Nodes:", num_nodes)
    print("Number of Edges:", num_edges)
    print("All Nodes:", all_nodes)
    print("All Edges:", all_edges)
    return

In [ ]:
### 2D Spring Plot

import matplotlib.pyplot as plt

# Draw the graph using a spring layout (for undirected graphs)
plt.figure(figsize=(10, 4))
pos = nx.spring_layout(graph) # , seed=43
nx.draw(graph, pos, node_size=8, node_color="skyblue"edge_color="gray")
# with_labels=True, font_size=8, font_color="black"
plt.title("Vessel Tree Graph")
plt.show()

In [ ]:
### Later:
# Approach-1: Make Interactive 3D Plot

from mpl_toolkits.mplot3d import Axes3D

# Create a 3D plot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Get node positions from the graph (assuming you have the coordinates in the nodes' attributes)
node_positions = {node: graph.nodes[node]['pos'] for node in graph.nodes}

# Draw nodes
for node, pos in node_positions.items():
    ax.scatter(pos[0], pos[1], pos[2], color='skyblue', s=100, label=node)

# Draw edges
for edge in graph.edges:
    pos1 = node_positions[edge[0]]
    pos2 = node_positions[edge[1]]
    ax.plot([pos1[0], pos2[0]], [pos1[1], pos2[1]], [pos1[2], pos2[2]], color='gray')

# Set axis labels
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

# Add legend
ax.legend()

# Show the interactive 3D plot
plt.show()

In [6]:
# Create graph/edges/neighbours/...

import numpy as np
from skimage.morphology import skeletonize_3d
import networkx as nx
import plotly.graph_objects as go  # pip install plotly
from plotly.subplots import make_subplots

# Step 2: Convert skeletonized image to a graph representation
graph = nx.Graph()
indices = np.array(np.where(skeleton))
num_points = indices.shape[1]
for i in range(num_points): # 20,100 # num_points
    point = tuple(indices[:, i])
    neighbors = []
    for offset in np.ndindex((3, 3, 3)):
        neighbor = tuple(indices[:, i] + np.array(offset) - 1)
        if neighbor != point and neighbor in graph:
            neighbors.append(neighbor)
    graph.add_node(point)
    graph.add_edges_from((point, n) for n in neighbors)

In [7]:
print_stats(graph)

NameError: name 'print_stats' is not defined

In [ ]:
### Remove all leafs?!!!!! x

# Step 3: Simplify the graph by removing intermediate nodes that lie on a single branch

leaf_nodes = [node for node, degree in graph.degree if degree == 1]

while len(leaf_nodes) > 1:
    graph.remove_nodes_from(leaf_nodes)
    leaf_nodes = [node for node, degree in graph.degree if degree == 1]

In [8]:
# Remove degree-2 nodes*

while True:
    # Find nodes with degree 2
    degree_two_nodes = [node for node, degree in graph.degree if degree == 2]
    
    # If there are no intermediate nodes to remove, break the loop
    if not degree_two_nodes:
        break
    
    # Remove the intermediate nodes
    for node in degree_two_nodes:
        neighbors = list(graph.neighbors(node))
        if len(neighbors) == 2:
            graph.add_edge(neighbors[0], neighbors[1])
        graph.remove_node(node)
        
# [node for node in graph if node[0] == 213 and node[1] == 229]

In [ ]:
print_stats(graph)

In [8]:
# Create edge 'edge_length' property

for edge in graph.edges:
    node1, node2 = edge
    x1, y1, z1 = node1
    x2, y2, z2 = node2
    edge_length = np.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)
    graph.edges[edge]['edge_length'] = edge_length

In [ ]:
# for indx, edge in enumerate(graph.edges):
#     print(graph.edges[edge])
#     if indx==0: break

# for indx, node in enumerate(graph.nodes):
#     print(graph.nodes[node])
#     if indx==0: break

In [9]:
# Function: Plot Interactive 3D*

def plot_3D(graph):
    fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'scatter3d'}]])
    x, y, z = zip(*graph.nodes)
    
    leaf_nodes = [node for node, degree in graph.degree if degree == 1]
    
    node_trace = go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode='markers',
        marker=dict(
            size=2,
            color=['red' if node in leaf_nodes else 'blue' for node in graph.nodes()],
            opacity=0.8
        )
    )
    fig.add_trace(node_trace)

    for edge in graph.edges():
        x0, y0, z0 = edge[0]
        x1, y1, z1 = edge[1]
        
        edge_length = graph.edges[edge]['edge_length']
        edge_text = f'{edge_length:.2f}'  # Display the edge length as text
        
        edge_trace = go.Scatter3d(
            x=[x0, x1],
            y=[y0, y1],
            z=[z0, z1],
            mode='lines',
            line=dict(color='orange' if graph.edges[edge]['edge_length'] < 20 else 'gray',
                      width=3),
            hoverinfo='text',
            text=edge_text
        )
        fig.add_trace(edge_trace)

    fig.update_layout(  
        scene=dict(
        # Set background color to transparent or any color you prefer
        bgcolor='rgba(0,0,0,0)',
        
        # Hide axes
        xaxis=dict(showbackground=False, showgrid=False, zeroline=False, showline=False),
        yaxis=dict(showbackground=False, showgrid=False, zeroline=False, showline=False),
        zaxis=dict(showbackground=False, showgrid=False, zeroline=False, showline=False)
        )
    )
    
    fig.update_layout(
        margin=dict(l=0, r=0, b=0, t=0),
        scene=dict(
            xaxis=dict(range=[0, max(x) + 1]),
            yaxis=dict(range=[0, max(y) + 1]),
            zaxis=dict(range=[0, max(z) + 1])
        )
    )

    fig.show()
    return

In [10]:
import time
StartTime = time.time()
print('Plotting the 3D Graph ..')
plot_3D(graph)
print(f'Elapsed: {time.time()-StartTime:5.2f}')

Plotting the 3D Graph ..


Elapsed:  2.62


In [ ]:
# list(graph.nodes).index((404, 136, 313))
# list(graph.nodes)[0]

In [ ]:
# indices2 = np.array(np.where(skeleton))
# indices2[:, 0]
# tuple(indices2[:, 0])
# list(graph.nodes).index((49, 268, 201))
# dict(graph.nodes)#[(49, 268, 201)]
# list(graph.nodes)[272]

In [ ]:
### Doesn't work for undirected graphs!

cycles = list(nx.simple_cycles(graph))
print("Cycles found:")
for cycle in cycles:
    print(cycle)

In [ ]:
### Bad code: Finds only 2-tuples of cycles, and oblivious of root, and maxcyclesize=3

# graph.size()

def find_cycles(graph):
    visited = set()
    cycles = []

    def dfs(node, parent):
        visited.add(node)
        for neighbor in graph.neighbors(node):
            if neighbor == parent:
                continue
            if neighbor in visited:
                cycles.append([neighbor, node])
            else:
                dfs(neighbor, node)

    for node in graph.nodes():
        if node not in visited:
            dfs(node, None)

    return cycles

cycles = find_cycles(graph)
print("Cycles found:")
for cycle in cycles:
    print(cycle)

In [ ]:
def find_cycles(graph, root_node):
    cycles = []

    def dfs(node, visited, path):
        visited.add(node)
        path.append(node)

        if len(path) >= 2 and path[0] in graph.neighbors(node):
            cycles.append(path[:3])  # Only consider cycles up to length 3

        if len(path) < 3:
            for neighbor in graph.neighbors(node):
                if neighbor not in visited and neighbor != root_node:
                    dfs(neighbor, visited.copy(), path.copy())

    dfs(root_node, set(), [])

    return cycles

# Assuming you already have an undirected graph named 'graph' and a known root node 'root_node'
cycles = find_cycles(graph, list(graph.nodes)[271])
print("Cycles found:")
for cycle in cycles:
    print(cycle)

In [ ]:
# Adjacency-2-Graph

# Given adjacency matrix as a dictionary
adjacency_dict = {
    1: [2, 3],
    2: [1, 4, 3],
    3: [1, 5, 2],
    4: [2],
    5: [3]
}

# Create a graph
G = nx.Graph()

# Add nodes and edges based on the adjacency dictionary
for node, neighbors in adjacency_dict.items():
    G.add_node(node)
    for neighbor in neighbors:
        G.add_edge(node, neighbor)

# Print the edges of the created graph
print("Edges of the graph:")
for edge in G.edges():
    print(edge)

In [ ]:
# Cycle-3: Good-Approach-8! Using adjacency matrix and without netwrokx: Creates 10 duplicates!

def find_length_3_cycles(graph, node, visited, parent=None, depth=0):
    visited[node] = True

    for neighbor in graph[node]:
        if not visited[neighbor]:
            find_length_3_cycles(graph, neighbor, visited, node, depth + 1)
        elif depth >= 2 and neighbor != parent:
            print("Length-3 cycle:", node, "->", neighbor, "->", parent)

    visited[node] = False

# Assuming your graph is represented as an adjacency list
graph = {
    1: [2, 3],
    2: [1, 4, 3],
    3: [1, 5, 2],
    4: [2],
    5: [3]
}

visited = {node: False for node in graph}
for node in graph:
    find_length_3_cycles(graph, node, visited)


In [ ]:
### Cycle-3: Approach-9! Bare-basic checking. Works great. Has 3! duplicates.

# Find all length-3 cycles
length_3_cycles = []
for node in G.nodes():
    neighbors = list(G.neighbors(node))
    for neighbor1 in neighbors:
        for neighbor2 in neighbors:
            if neighbor1 != neighbor2 and G.has_edge(neighbor1, neighbor2):
                length_3_cycles.append([node, neighbor1, neighbor2])

# Print the length-3 cycles
for cycle in length_3_cycles:
    print(cycle)

In [ ]:
# Approach-9 with duplicates removed

# Find all length-3 cycles without duplicates
length_3_cycles = set()
for node in graph.nodes():
    neighbors = list(graph.neighbors(node))
    for neighbor1 in neighbors:
        for neighbor2 in neighbors:
            if neighbor1 != neighbor2 and graph.has_edge(neighbor1, neighbor2):
                cycle = tuple(sorted([node, neighbor1, neighbor2]))  # Sorting to avoid permutations
                length_3_cycles.add(cycle)

# Convert the cycles back to lists for printing
length_3_cycles = [list(cycle) for cycle in length_3_cycles]

# Print the length-3 cycles without duplicates
for cycle in length_3_cycles:
    print(cycle)

In [ ]:
EdgesInCycles = [[L[i], L[(i+1)%3]] for L in length_3_cycles for i in [0, 1, 2]]
EdgesInCycles

In [ ]:
graph.edges[EdgesInCycles[2]]

In [ ]:
# 6 % 3
# 2+1 % 3 = 3!

In [ ]:
### Experiments

# KTHCYCLE_NODES = length_3_cycles[0]
# KTHCYCLE_NODES
# # list(graph.edges)[0] in length_3_cycles
# # length_3_cycles[0]
# ITHEDGE_NODES = list(list(graph.edges)[0])
# ITHEDGE_NODES
# EdgeIsInCycle = all(item in KTHCYCLE_NODES for item in ITHEDGE_NODES)
# EdgeIsInCycle

In [ ]:
# Plot Interactive 3D-2!

fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'scatter3d'}]])
x, y, z = zip(*graph.nodes)

leaf_nodes = [node for node, degree in graph.degree if degree == 1]

node_trace = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=2,
        color=['red' if node in leaf_nodes else 'blue' for node in graph.nodes()],
        opacity=0.4 #0.8
    )
)
fig.add_trace(node_trace)

for edge in graph.edges():
    x0, y0, z0 = edge[0]
    x1, y1, z1 = edge[1]
    
    edge_length = graph.edges[edge]['edge_length']
    edge_text = f'{edge_length:.2f}'  # Display the edge length as text
    
    edge_is_in_a_cycle = [edge[0],edge[1]] in EdgesInCycles or [edge[1],edge[0]] in EdgesInCycles
    
    edge_trace = go.Scatter3d(
        x=[x0, x1],
        y=[y0, y1],
        z=[z0, z1],
        mode='lines',
        line=dict(color='rgba(255, 0, 0, 0.8)' if edge_is_in_a_cycle else 'rgba(128, 128, 128, 0.4)',
            width=3),
        hoverinfo='text',
        text=edge_text
    )
    fig.add_trace(edge_trace)

fig.update_layout(
    margin=dict(l=0, r=0, b=0, t=0),
    scene=dict(
        xaxis=dict(range=[0, max(x) + 1]),
        yaxis=dict(range=[0, max(y) + 1]),
        zaxis=dict(range=[0, max(z) + 1])
    )
)

fig.show()  

In [ ]:
# set([1,2]) == set([2,1])
set([1,2]) in set([1,2,3])

![Alt text](image.png)

In [ ]:
# True networkx adjacency matrix

# Get the adjacency matrix
adj_matrix = nx.adjacency_matrix(G)

# Convert the sparse matrix to a dense NumPy array
adj_matrix_dense = adj_matrix.toarray()

# Print the adjacency matrix
print(adj_matrix_dense)

In [10]:
from mayavi import mlab
import numpy as np

# Create a figure
mlab.figure()

# Create a meshgrid for your 3D object
x, y, z = np.where(graph == 1)

# Plot the 3D object as a surface
mlab.points3d(x, y, z, scale_factor=1, color=(0, 0, 1), mode='cube')

mlab.show()

ValueError: not enough values to unpack (expected 3, got 1)